<a href="https://colab.research.google.com/github/matheuspercario/TT007-AnaliseDadosPython/blob/master/exercicios/Aula5-POO/liarsdice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício - Liars Dice


Criado por: Matheus Bruder

## Instruções de jogo

No Liers’ dice, cada jogador recebe 5 dados. Em cada rodada os jogadores lançam os seus dados, mantendo suas faces escondidas dos demais jogadores.
Cada jogador então escolhe uma face (1 a 6) e da um palpite de quantas faces iguais a esta há na mesa.
O próximo a jogar tem três alternativas:
  * _Escolhe uma face e da um palpite, que deve ser necessariamente maior que o palpite anterior e passa para o próximo jogador._
  * _Chama o jogador anterior de mentiroso, neste caso abrem-se os dados e verifica-se o palpite corrente é verdadeiro. Se for o jogador atual perde um dado, senão o jogador anterior perde um dado._
  * _Concorda com o palpite do jogador anterior. Neste caso, abrem-se os dados e se o palpite estiver correto, o jogador anterior perde dois dados._

Ganha o último jogador com dados na mesa.

### Informações extras
* O jogo deve ser para um usuário contra até 4 jogadores controlados pelo computador;
* Use Objetos;
* Considere usar probabilidades para que um jogador do PC, de um palpite, chame de mentiroso ou concorde com o jogador anterior;
* Use diferentes probabilidade para diferentes jogadores para simular diferentes comportamentos;
* É possível utilizar-se de alguma heurística para melhorar os jogadores do PC?
* Quem fica mais tempo na mesa, um jogador com heurística ou um jogador aleatório?

---

# Preparação das classes

In [1]:
# Iniciar códigos aqui

In [113]:
import random
import time

from os import system
from sys import exit

In [159]:
class Player:

    dice = []


    def __init__(self, name, num_die=5):
        self._name = name
        self._num_die = num_die


    @property
    def name(self):
        """Return the name."""
        return self._name


    @name.setter
    def name(self, name):
        """Set the name."""
        name.strip().capitalize()
        if len(name) > 25:
            raise ValueError(f'Name must be less than 25 characters')

        self._name = name


    @property
    def num_die(self):
        """Return the number of die."""
        return self._num_die


    @num_die.setter
    def num_die(self, num_die):
        """Set the number of dice."""
        self._num_die = num_die

    
    def roll_dice(self):
        self.__class__.dice.clear()
        for i in range(self._num_die):
            self.__class__.dice.append(random.randrange(1, 7)) 


    def show_hand(self):
        string = ''
        for d in self.__class__.dice:
            if d == 0:
                continue
            string += f'[{d}] '
        print(f'{self.name}, your hand: {string}')


    def remove_die(self):
        self._num_die -= 1




    def __repr__(self):
        """Return string representation for repr()."""
        return (f'(Nome={self.name}, Numero_dados={self.num_die}, Dados={self.__class__.dice})')
    

In [160]:
class NPC(Player):

    def __init__(self, name, num_die=5):
        super().__init__(name, num_die=5)


    def bet(self, old_bet, num_dice):
        new_bet = [0, 0]
        while True:
            b0 = random.randrange(1, num_dice + 1)
            b1 = random.randrange(1, 7)
            new_bet = [int(b0), int(b1)]
            if old_bet[0] >= new_bet[0]:
                print('Aposta inválida, tente novamente.')
                continue
            break
        return new_bet
    

    def __repr__(self):
        """Return string representation for repr()."""
        return (self.__class__.__name__ + '\t-> ' + super().__repr__())

In [161]:
class Human(Player):

    def __init__(self, name, num_die=5):
        super().__init__(name, num_die=5)


    def bet(self, old_bet, num_dice):
        new_bet = [0, 0]
        while True:
            new_bet = input('Faça sua aposta! [Número de dados] [Valor do dado]\n: ').split()
            if len(new_bet) == 2 and new_bet[0].isdigit() and new_bet[1].isdigit():
                new_bet = [int(new_bet[0]), int(new_bet[1])]
                if old_bet[0] >= new_bet[0] or new_bet[0] > num_dice or new_bet[1] > 6 or new_bet[1] < 1:
                    print('Aposta inválida, tente novamente.')
                    continue
                break
        return new_bet


    def __repr__(self):
        """Return string representation for repr()."""
        return (self.__class__.__name__ + '\t-> ' + super().__repr__())

# Funções & dados auxiliares

In [162]:
NAMES = 'Josie Henry Catherine Carl Henrietta Joseph Alistair Conrad Gary Bethany Susan Kirstie Maverick Mason Sophie Claire'.split()

# Jogo começa aqui

In [163]:
player_name = input('Qual é seu nome?\n: ').strip().capitalize()
num_players = int(input('Quantos jogadores?\n: '))

print('\n\n\n')

players = []
for n in range(num_players - 1):
    name = NAMES[random.randint(0, len(NAMES)-1)]
    NAMES.remove(name)
    players.append(NPC(name))

players.append(Human(player_name))


gameContinues = True
nextPlayer = random.randrange(1, len(players) + 1)  # definir quem começa
# nextPlayer = 2


# Inicio de jogo
while gameContinues:
    # definir dados da rodada
    for p in players:
        p.roll_dice()
        
    current_player = players[nextPlayer]
    current_bet = [0, 0]

    if current_player.__class__.__name__ == 'Human':
        current_player.show_hand()

        n_dice = 0
        # Contar numero de dados em jogo
        for p in players:
            n_dice += p.num_die
        print(f'Existem {n_dice - current_player.num_die} outros dados na mesa.\n')
    
    
    print(f'{current_player.name} começa. Por favor, faça a primeira aposta.\n')

    current_bet = current_player.bet(current_bet, n_dice)
    print(f'{current_player.name} apostou que existem {str(current_bet[0])} dados com valor [{str(current_bet[1])}] na mesa.\n')

    
    # Continuando os rounds
    status = True
    while status:
        nextPlayer = (nextPlayer + 1) % len(players)

        current_player = players[nextPlayer]

        # Mostrando opções ao jogador
        if current_player.__class__.__name__ == 'Human':
            current_player.show_hand()

            n_dice = 0
            # Contar numero de dados em jogo
            for p in players:
                n_dice += p.num_die
            print(f'Existem {n_dice - current_player.num_die} outros dados na mesa.\n')

        choose = choose()

    gameContinues = False


Qual é seu nome?
: Suetam
Quantos jogadores?
: 3




Gary começa. Por favor, faça a primeira aposta.

Gary apostou que existem 11 dados com valor [1] na mesa.



In [32]:
class Lairsdice:
    
    def __init__(self, num_players):
        self._num_players = num_players
        self._gameContinues = gameContinues


    @property
    def num_players(self):
        """Return the number of players."""
        return self._num_players

    
    @num_players.setter
    def num_players(self, num_players):
        """Set the number of players."""
        if num_players > 5 or num_players < 2:
            raise ValueError(f'Name must be less than 25 characters')

        self._num_players = num_players

    def create_npcs(self):
        pass